In [1]:
import pandas as pd
import numpy as np

In [2]:
FAST, SLOW = 5, 15
data = pd.read_csv('SP500 stock OHLC.csv', header=0, index_col=0, parse_dates=[0])
data

,Open,High,Low,Close,Adj Close,Volume,symbol,industry
Date,,,,,,,,
2010-12-01,25.500715,25.858370,25.472103,25.836910,23.462572,5770804.0,A,Health Care
2010-12-02,25.736767,26.351931,25.672388,26.266094,23.852320,4237338.0,A,Health Care
2010-12-03,26.108727,26.888412,26.094421,26.809729,24.345999,4565449.0,A,Health Care
2010-12-06,26.680973,26.752504,26.502146,26.559372,24.118643,2309216.0,A,Health Care
2010-12-07,26.888412,27.095852,26.824034,26.881260,24.410952,5754727.0,A,Health Care
...,...,...,...,...,...,...,...,...
2020-12-23,222.630005,224.910004,222.610001,224.240005,224.240005,4983900.0,BRK-B,Financials
2020-12-24,224.600006,227.130005,224.210007,226.529999,226.529999,2387000.0,BRK-B,Financials
2020-12-28,228.089996,230.149994,227.360001,228.410004,228.410004,3505500.0,BRK-B,Financials


In [3]:
def SMA_ratio(data, fast, slow):
    data['SMA_fast'] = data.groupby('symbol').Close.transform(lambda x: x.rolling(window=fast).mean())
    data['SMA_slow'] = data.groupby('symbol').Close.transform(lambda x: x.rolling(window=slow).mean())
    data['SMA_ratio'] = data.SMA_fast/data.SMA_slow

In [4]:
def SMA_volume_ratio(data, fast, slow):
    data['SMA_fast_volume'] = data.groupby('symbol').Volume.transform(lambda x: x.rolling(window=fast).mean())
    data['SMA_slow_volume'] = data.groupby('symbol').Volume.transform(lambda x: x.rolling(window=slow).mean())
    data['SMA_volume_ratio'] = data.SMA_fast_volume/data.SMA_slow_volume

In [5]:
def Wilder(data, period):
    # Wilder's Smoothing
    start = np.where(~np.isnan(data))[0][0]
    wilder = np.array([np.nan] * len(data))
    wilder[start+period-1] = data[start:(start+period)].mean()
    for i in range(start+period, len(data)):
        wilder[i] = (wilder[i-1] * (period-1) + data[i])/period
    return wilder

In [6]:
def ATR(data, fast, slow):
    # Average True Range
    data['prev_close'] = data.groupby('symbol').Close.shift(1)
    data['TR'] = np.maximum((data.High - data.Low),
                np.maximum(abs(data.High - data.prev_close),
                abs(data.prev_close - data.Close)))
    for ticker in data.symbol.unique():
        temp = data[data.symbol == ticker].copy()
        data.loc[data.symbol == ticker, 'ATR_fast'] = Wilder(temp['TR'], fast)
        data.loc[data.symbol == ticker, 'ATR_slow'] = Wilder(temp['TR'], slow)
    data['ATR_ratio'] = data.ATR_fast/data.ATR_slow

In [7]:
def ADX(data, fast, slow):
    # Average Directional Index
    data['prev_high'] = data.groupby('symbol').High.shift(1)
    data['prev_low'] = data.groupby('symbol').Low.shift(1)
    data['+DM'] = np.where(~np.isnan(data.prev_high),
                           np.where((data['High'] > data['prev_high']) & 
         (((data['High'] - data['prev_high']) > (data['prev_low'] - data['Low']))), 
                                                                  data['High'] - data['prev_high'], 
                                                                  0),np.nan)

    data['-DM'] = np.where(~np.isnan(data.prev_low),
                               np.where((data['prev_low'] > data['Low']) & 
             (((data['prev_low'] - data['Low']) > (data['High'] - data['prev_high']))), 
                                        data['prev_low'] - data['Low'], 
                                        0),np.nan)

    for ticker in data['symbol'].unique():
        temp = data[data.symbol == ticker].copy()
        data.loc[data.symbol==ticker,'+DM_fast'] = Wilder(temp['+DM'], fast)
        data.loc[data.symbol==ticker,'-DM_fast'] = Wilder(temp['-DM'], fast)
        data.loc[data.symbol==ticker,'+DM_slow'] = Wilder(temp['+DM'], slow)
        data.loc[data.symbol==ticker,'-DM_slow'] = Wilder(temp['-DM'], slow)

    data['+DI_fast'] = (data['+DM_fast']/data['ATR_fast'])*100
    data['-DI_fast'] = (data['-DM_fast']/data['ATR_fast'])*100
    data['+DI_slow'] = (data['+DM_slow']/data['ATR_slow'])*100
    data['-DI_slow'] = (data['-DM_slow']/data['ATR_slow'])*100

    data['DX_fast'] = (np.round(abs(data['+DI_fast'] - data['-DI_fast'])/(data['+DI_fast'] + data['-DI_fast']) * 100))

    data['DX_slow'] = (np.round(abs(data['+DI_slow'] - data['-DI_slow'])/(data['+DI_slow'] + data['-DI_slow']) * 100))

    for ticker in data['symbol'].unique():
        temp = data[data.symbol == ticker].copy()
        data.loc[data.symbol==ticker,'ADX_fast'] = Wilder(temp['DX_fast'], fast)
        data.loc[data.symbol==ticker,'ADX_slow'] = Wilder(temp['DX_slow'], slow)

In [8]:
def StochOsc(data, fast, slow):
    data['Lowest_fast'] = data.groupby('symbol')['Low'].transform(lambda x: x.rolling(window = fast).min())
    data['High_fast'] = data.groupby('symbol')['High'].transform(lambda x: x.rolling(window = fast).max())
    data['Lowest_slow'] = data.groupby('symbol')['Low'].transform(lambda x: x.rolling(window = slow).min())
    data['High_slow'] = data.groupby('symbol')['High'].transform(lambda x: x.rolling(window = slow).max())

    data['Stochastic_fast'] = ((data['Close'] - data['Lowest_fast'])/(data['High_fast'] - data['Lowest_fast']))*100
    data['Stochastic_slow'] = ((data['Close'] - data['Lowest_slow'])/(data['High_slow'] - data['Lowest_slow']))*100

    data['Stochastic_%D_fast'] = data['Stochastic_fast'].rolling(window = fast).mean()
    data['Stochastic_%D_slow'] = data['Stochastic_slow'].rolling(window = slow).mean()

    data['Stochastic_Ratio'] = data['Stochastic_%D_fast']/data['Stochastic_%D_slow']

In [9]:
def RSI(data, fast, slow):
    data['Diff'] = data.groupby('symbol')['Close'].transform(lambda x: x.diff())
    data['Up'] = data['Diff']
    data.loc[(data['Up']<0), 'Up'] = 0

    data['Down'] = data['Diff']
    data.loc[(data['Down']>0), 'Down'] = 0 
    data['Down'] = abs(data['Down'])

    data['avg_fast_up'] = data.groupby('symbol')['Up'].transform(lambda x: x.rolling(window=fast).mean())
    data['avg_fast_down'] = data.groupby('symbol')['Down'].transform(lambda x: x.rolling(window=fast).mean())

    data['avg_slow_up'] = data.groupby('symbol')['Up'].transform(lambda x: x.rolling(window=slow).mean())
    data['avg_slow_down'] = data.groupby('symbol')['Down'].transform(lambda x: x.rolling(window=slow).mean())

    data['RS_fast'] = data['avg_fast_up'] / data['avg_fast_down']
    data['RS_slow'] = data['avg_slow_up'] / data['avg_slow_down']

    data['RSI_fast'] = 100 - (100/(1+data['RS_fast']))
    data['RSI_slow'] = 100 - (100/(1+data['RS_slow']))

    data['RSI_ratio'] = data['RSI_fast']/data['RSI_slow']

In [10]:
def MACD(data, fast, slow):
    data['fast_Ewm'] = data.groupby('symbol')['Close'].transform(lambda x: x.ewm(span=fast, adjust=False).mean())
    data['slow_Ewm'] = data.groupby('symbol')['Close'].transform(lambda x: x.ewm(span=slow, adjust=False).mean())
    data['MACD'] = data['fast_Ewm'] - data['slow_Ewm']

In [11]:
def RateofChange(data, slow):
    data['RC'] = data.groupby('symbol')['Close'].transform(lambda x: x.pct_change(periods = slow))

In [12]:
def get_indicators(data, fast, slow):
    SMA_ratio(data, fast, slow)
    SMA_volume_ratio(data, fast, slow)
    ATR(data, fast, slow)
    ADX(data, fast, slow)
    StochOsc(data, fast, slow)
    RSI(data, fast, slow)
    MACD(data, fast, slow)
    RateofChange(data, slow)

In [13]:
get_indicators(data, FAST, SLOW)
data.to_csv("SP500 stock TI.csv")
print(data)

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2010-12-01   25.500715   25.858370   25.472103   25.836910   23.462572   
2010-12-02   25.736767   26.351931   25.672388   26.266094   23.852320   
2010-12-03   26.108727   26.888412   26.094421   26.809729   24.345999   
2010-12-06   26.680973   26.752504   26.502146   26.559372   24.118643   
2010-12-07   26.888412   27.095852   26.824034   26.881260   24.410952   
...                ...         ...         ...         ...         ...   
2020-12-23  222.630005  224.910004  222.610001  224.240005  224.240005   
2020-12-24  224.600006  227.130005  224.210007  226.529999  226.529999   
2020-12-28  228.089996  230.149994  227.360001  228.410004  228.410004   
2020-12-29  230.500000  231.100006  229.100006  229.570007  229.570007   
2020-12-30  229.539993  230.589996  229.080002  229.649994  229.649994   

               Volume symbol     indu